# ML - Final Project 
Author: Gaurav Kuwar

### Ensure Terraform is installed

Use homebrew on mac to install

In [1]:
terraform --version

Terraform v1.11.4
on darwin_arm64

Your version of Terraform is out of date! The latest version
is 1.12.0. You can update by downloading from https://developer.hashicorp.com/terraform/install


### Configure python env and install

In [2]:
export PROJECT_ROOT=$(pwd)
echo $PROJECT_ROOT

/Users/gauravk/Documents/Classes/Spring 2025/ML_SYS/proj/mlsysops-cms


In [3]:
python -m venv .venv
source .venv/bin/activate
pip install -r requirements.txt

(.venv) 
(.venv) 


### Terraform Configure

Update clouds.yaml `REDACTED_UNIQUE_ID` and `REDACTED_SECRET` before proceeding.

In [4]:
cp config/clouds.yaml iac/tf/kvm/clouds.yaml

(.venv) 


In [5]:
cd iac/tf/kvm/
export PATH=/work/.local/bin:$PATH # unset env variables
ls clouds.yaml # check this exists

(.venv) 
(.venv) 
clouds.yaml
(.venv) 


In [6]:
terraform init

Initializing the backend...
Initializing provider plugins...
- Reusing previous version of terraform-provider-openstack/openstack from the dependency lock file
- Using previously-installed terraform-provider-openstack/openstack v1.51.1

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
(.venv) 


In [7]:
# using my netid because of late submission
export TF_VAR_suffix=project26_gk2657
export TF_VAR_key=id_rsa_chameleon

(.venv) 
(.venv) 
(.venv) 


In [8]:
terraform validate

Success! The configuration is valid.

(.venv) 


In [9]:
terraform plan

data.openstack_networking_secgroup_v2.allow_8081: Reading...
data.openstack_networking_secgroup_v2.allow_8000: Reading...
data.openstack_networking_secgroup_v2.allow_8080: Reading...
data.openstack_networking_network_v2.sharednet1: Reading...
data.openstack_networking_secgroup_v2.allow_http_80: Reading...
data.openstack_networking_secgroup_v2.allow_ssh: Reading...
data.openstack_networking_secgroup_v2.allow_9001: Reading...
data.openstack_networking_secgroup_v2.allow_9090: Reading...
data.openstack_networking_subnet_v2.sharednet1_subnet: Reading...
data.openstack_networking_secgroup_v2.allow_ssh: Read complete after 1s [id=47860acf-9269-4d26-84b1-363824f7e0a2]
data.openstack_networking_secgroup_v2.allow_9001: Read complete after 1s [id=ad2912c0-caa9-49e0-b431-b9b3953b91b3]
data.openstack_networking_secgroup_v2.allow_8081: Read complete after 1s [id=9806d71f-f0bd-4c5d-8c48-f32db3094575]
data.openstack_networking_secgroup_v2.allow_9090: Read complete after 1s [id=d6736d54-d9a4-431c-a5b5-

In [10]:
terraform apply -auto-approve

data.openstack_networking_secgroup_v2.allow_8081: Reading...
data.openstack_networking_secgroup_v2.allow_9090: Reading...
data.openstack_networking_secgroup_v2.allow_ssh: Reading...
data.openstack_networking_subnet_v2.sharednet1_subnet: Reading...
data.openstack_networking_secgroup_v2.allow_8000: Reading...
data.openstack_networking_secgroup_v2.allow_http_80: Reading...
data.openstack_networking_secgroup_v2.allow_8080: Reading...
data.openstack_networking_secgroup_v2.allow_9001: Reading...
data.openstack_networking_network_v2.sharednet1: Reading...
data.openstack_networking_secgroup_v2.allow_ssh: Read complete after 1s [id=47860acf-9269-4d26-84b1-363824f7e0a2]
data.openstack_networking_secgroup_v2.allow_8081: Read complete after 1s [id=9806d71f-f0bd-4c5d-8c48-f32db3094575]
data.openstack_networking_secgroup_v2.allow_8080: Read complete after 1s [id=e6eff5db-c96d-4e3d-be49-1ade281f4a7a]
data.openstack_networking_subnet_v2.sharednet1_subnet: Read complete after 1s [id=06c725cd-feef-4bf3-

In [11]:
cd "$PROJECT_ROOT"

(.venv) 


### Ansible

In [13]:
export ANSIBLE_CONFIG=ansible.cfg
export ANSIBLE_ROLES_PATH=roles

(.venv) 
(.venv) 


In [14]:
ssh-add ~/.ssh/id_rsa_chameleon # add key to agent
# ssh-add -l # then check

Identity added: /Users/gauravk/.ssh/id_rsa_chameleon (gauravk@Gauravs-MacBook-Pro.local)
(.venv) 
(.venv) 


Update `ansible.cfg` with floating IP and then copy

In [15]:
cp config/ansible.cfg iac/ansible/ansible.cfg
cd iac/ansible # change to ansible dir

(.venv) 
(.venv) 


In [16]:
ansible -i inventory.yaml all -m ping

node3 | SUCCESS => {
    "changed": false,
    "ping": "pong"
}
node1 | SUCCESS => {
    "changed": false,
    "ping": "pong"
}
node2 | SUCCESS => {
    "changed": false,
    "ping": "pong"
}
(.venv) 


In [17]:
ansible-playbook -i inventory.yaml general/hello_host.yml # test it works


PLAY [Hello, world - use Ansible to run a command on each host] ****************

TASK [Run hostname command] ****************************************************
changed: [node1]
changed: [node3]
changed: [node2]

TASK [Show hostname output] ****************************************************
ok: [node1] => 
  msg: The hostname of node1 is node1-mlops-project26-gk2657
ok: [node2] => 
  msg: The hostname of node2 is node2-mlops-project26-gk2657
ok: [node3] => 
  msg: The hostname of node3 is node3-mlops-project26-gk2657

PLAY RECAP *********************************************************************
node1                      : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
node2                      : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
node3                      : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   

(.venv) 


### Install and Setup Kubernetes Cluster

In [18]:
ansible-playbook -i inventory.yaml --become --become-user=root pre_k8s/pre_k8s_configure.yml


PLAY [Disable and Mask firewalld] **********************************************

TASK [Gathering Facts] *********************************************************
ok: [node3]
ok: [node1]
ok: [node2]

TASK [Stop firewalld service] **************************************************
changed: [node1]
changed: [node2]
changed: [node3]

TASK [Mask firewalld service] **************************************************
changed: [node1]
changed: [node3]
changed: [node2]

PLAY [Set up insecure registry for Docker] *************************************

TASK [Gathering Facts] *********************************************************
ok: [node1]
ok: [node3]
ok: [node2]

TASK [Ensure /etc/docker directory exists] *************************************
changed: [node1]
changed: [node2]
changed: [node3]

TASK [Create /etc/docker/daemon.json if not exists] ****************************
changed: [node1]
changed: [node2]
changed: [node3]

TASK [Configure Docker daemon.json for insecure registry] *********

In [19]:
ansible-playbook -i inventory.yaml --become --become-user=root k8s/k8s_install.yml


PLAY [Initial Stuff] ***********************************************************

TASK [Gathering Facts] *********************************************************
ok: [node2]
ok: [node3]
ok: [node1]

TASK [Updates] *****************************************************************
changed: [node3]
changed: [node2]
changed: [node1]

TASK [Disable SWAP] ************************************************************
changed: [node2]
changed: [node1]
changed: [node3]

TASK [Disable SWAP in fstab] ***************************************************
ok: [node1]
ok: [node2]
ok: [node3]

PLAY [Install containerd using official GPG + keyrings method] *****************

TASK [Gathering Facts] *********************************************************
ok: [node1]
ok: [node2]
ok: [node3]

TASK [Install dependencies] ****************************************************
ok: [node1]
ok: [node2]
ok: [node3]

TASK [Create keyrings directory] ***********************************************
ok: [node3]
ok: 

In [20]:
ansible-playbook -i inventory.yaml --become --become-user=root k8s/k8s_master.yml


PLAY [master] ******************************************************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Initialize the cluster] **************************************************
changed: [node1]

TASK [Save output to log file] *************************************************
changed: [node1]

TASK [Create .kube directory] **************************************************
changed: [node1]

TASK [Copy admin.conf to User's kube config] ***********************************
changed: [node1]

PLAY [Configure and apply Flannel] *********************************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Apply Flannel] ***********************************************************
changed: [node1]

TASK [Get specific interface used to reach] ************************************
ok: [node1]

TASK [Show interface] ************************************

In [21]:
ansible-playbook -i inventory.yaml --become --become-user=root k8s/k8s_worker.yml


PLAY [Configure Join Commands on Master Node] **********************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Retrieve Join Command] ***************************************************
changed: [node1]

TASK [Set Join Command] ********************************************************
ok: [node1]

PLAY [Join Worker Nodes] *******************************************************

TASK [Gathering Facts] *********************************************************
ok: [node3]
ok: [node2]

TASK [Enable TCP port 6443 (On Master) is able to connect from Worker;] ********
ok: [node3]
ok: [node2]

TASK [Join worker to cluster] **************************************************
changed: [node2]
changed: [node3]

PLAY RECAP *********************************************************************
node1                      : ok=3    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
node2                

In [22]:
ansible-playbook -i inventory.yaml --become --become-user=root post_k8s/post_k8s_configure.yml


PLAY [Configure DNS and Restart CoreDNS] ***************************************

TASK [Gathering Facts] *********************************************************
ok: [node3]
ok: [node2]
ok: [node1]

TASK [Set DNS for interface ens4] **********************************************
changed: [node2]
changed: [node1]
changed: [node3]

PLAY [Restart CoreDNS on node1] ************************************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Restart CoreDNS deployment] **********************************************
changed: [node1]

PLAY [Configure CoreDNS resolv.conf across all nodes] **************************

TASK [Gathering Facts] *********************************************************
ok: [node3]
ok: [node1]
ok: [node2]

TASK [Get CoreDNS ClusterIP (run only on master)] ******************************
changed: [node1]

TASK [Set CoreDNS IP as fact] **************************************************
ok: [node1]

#### Access the ArgoCD dashboard

`ssh -L 8888:127.0.0.1:8888 cc@A.B.C.D` on local

`kubectl port-forward svc/argocd-server -n argocd 8888:443` on node1

`https://127.0.0.1:8888/` on browser

#### Access the Argo Workflows dashboard

`ssh -L 2746:127.0.0.1:2746 cc@A.B.C.D` on local

`kubectl -n argo port-forward svc/argo-server 2746:2746` on node1

`https://127.0.0.1:2746/` on browser

In [23]:
cd "$PROJECT_ROOT"

(.venv) 


### Create block storage and attach to node1

- Manual step - create a block storage and attach to node1
- Run ONLY ONCE - if reloading nodes, but want presistant data, don't reformat

In [24]:
cd "$PROJECT_ROOT"
cd iac/ansible

(.venv) 
(.venv) 


In [25]:
ansible-playbook -i inventory.yaml post_k8s/block_configure.yml


PLAY [Partition Chameleon block volume for MLflow stack] ***********************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Ensure mount point exists] ***********************************************
changed: [node1]

TASK [Format block device if not already formatted] ****************************
ok: [node1]

TASK [Mount block device] ******************************************************
changed: [node1]

TASK [Ensure mount persists in /etc/fstab] *************************************
ok: [node1]

TASK [Create subdirectories for MinIO and Postgres] ****************************
ok: [node1] => (item=minio-data)
changed: [node1] => (item=postgres-data)
ok: [node1] => (item=data/kaggle)
ok: [node1] => (item=data/preprocessed)

PLAY RECAP *********************************************************************
node1                      : ok=6    changed=3    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   

(.venv)

### Manage applications with ArgoCD

In [26]:
cd "$PROJECT_ROOT"
cd iac/ansible

(.venv) 
(.venv) 


In [62]:
ansible-playbook -i inventory.yaml argocd/workflow_build_init.yml


PLAY [Run Argo Workflow from GitHub Repo] **************************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Clone or update mlsysops-cms-iac repo] ***********************************
changed: [node1]

PLAY [Run Argo Workflow from GitHub Repo] **************************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Clone or update mlsysops-cms-iac repo] ***********************************
ok: [node1]

TASK [Submit Argo Workflow] ****************************************************
changed: [node1]

TASK [Extract Workflow Name] ***************************************************
ok: [node1]

TASK [Wait for workflow to complete (success or fail)] *************************
changed: [node1]

TASK [Get final workflow result] ***********************************************
changed: [node1]

TASK [Display workflow phase] *********************************

In [57]:
ansible-playbook -i inventory.yaml argocd/argocd_add_platform.yml


PLAY [Deploy MLflow platform via ArgoCD & Helm with MinIO secret handling] *****

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Get ArgoCD admin password from Kubernetes secret] ************************
changed: [node1]

TASK [Decode ArgoCD admin password] ********************************************
changed: [node1]

TASK [Log in to ArgoCD] ********************************************************
ok: [node1]

TASK [Add repository to ArgoCD] ************************************************
changed: [node1]

TASK [Detect external IP starting with 10.56] **********************************
ok: [node1]

TASK [Ensure mlsysops-cms-platform namespace exists] ***************************
ok: [node1]

TASK [Create mlsysops-cms-platform namespace if missing] ***********************
changed: [node1]

TASK [Check if MinIO secret already exists] ************************************
ok: [node1]

TASK [Generate MinIO secret key] ******************

In [58]:
ansible-playbook -i inventory.yaml argocd/argocd_add_staging.yml


PLAY [Deploy mlsysops-cms Staging via ArgoCD & Helm] ***************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Get ArgoCD admin password] ***********************************************
changed: [node1]

TASK [Decode ArgoCD password] **************************************************
changed: [node1]

TASK [Login to ArgoCD] *********************************************************
ok: [node1]

TASK [Detect external IP starting with 10.56] **********************************
ok: [node1]

TASK [Create mlsysops-cms-staging namespace if missing] ************************
ok: [node1]

TASK [Check if ArgoCD app exists] **********************************************
ok: [node1]

TASK [Create ArgoCD Helm app if not exists] ************************************
changed: [node1]

TASK [Update ArgoCD Helm app if exists] ****************************************
skipping: [node1]

TASK [Display ArgoCD app status] *****************

In [59]:
ansible-playbook -i inventory.yaml argocd/argocd_add_canary.yml


PLAY [Deploy mlsysops-cms Canary via ArgoCD & Helm] ****************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Get ArgoCD admin password] ***********************************************
changed: [node1]

TASK [Decode ArgoCD password] **************************************************
changed: [node1]

TASK [Login to ArgoCD] *********************************************************
ok: [node1]

TASK [Detect external IP starting with 10.56] **********************************
ok: [node1]

TASK [Create mlsysops-cms-canary namespace if missing] *************************
ok: [node1]

TASK [Check if ArgoCD app exists] **********************************************
ok: [node1]

TASK [Create ArgoCD Helm app if not exists] ************************************
changed: [node1]

TASK [Update ArgoCD Helm app if exists] ****************************************
skipping: [node1]

TASK [Display ArgoCD app status] *****************

In [60]:
ansible-playbook -i inventory.yaml argocd/argocd_add_prod.yml


PLAY [Deploy mlsysops-cms Production via ArgoCD & Helm] ************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Get ArgoCD admin password] ***********************************************
changed: [node1]

TASK [Decode ArgoCD password] **************************************************
changed: [node1]

TASK [Login to ArgoCD] *********************************************************
ok: [node1]

TASK [Detect external IP starting with 10.56] **********************************
ok: [node1]

TASK [Create mlsysops-cms-production namespace if missing] *********************
ok: [node1]

TASK [Check if ArgoCD app exists] **********************************************
ok: [node1]

TASK [Create ArgoCD Helm app if not exists] ************************************
changed: [node1]

TASK [Update ArgoCD Helm app if exists] ****************************************
skipping: [node1]

TASK [Display ArgoCD app status] *****************

In [32]:
cd "$PROJECT_ROOT"
cp config/kaggle.json iac/ansible/argocd/kaggle.json
cd iac/ansible

(.venv) 
(.venv) 
(.venv) 


In [ ]:
ansible-playbook -i inventory.yaml argocd/workflow_templates_apply.yml


PLAY [Clone repo and apply specific Argo WorkflowTemplates] ********************

TASK [Gathering Facts] *********************************************************


#### Delete Infra

In [78]:
cd "$PROJECT_ROOT"

(.venv) 


In [79]:
cd iac/tf/kvm/

(.venv) 


In [80]:
terraform destroy -auto-approve

data.openstack_networking_secgroup_v2.allow_8080: Reading...
data.openstack_networking_secgroup_v2.allow_9090: Reading...
data.openstack_networking_secgroup_v2.allow_http_80: Reading...
data.openstack_networking_secgroup_v2.allow_8081: Reading...
data.openstack_networking_secgroup_v2.allow_ssh: Reading...
data.openstack_networking_secgroup_v2.allow_9001: Reading...
data.openstack_networking_secgroup_v2.allow_8000: Reading...
openstack_networking_network_v2.private_net: Refreshing state... [id=9a949a9d-e07f-4f7d-a32b-645f1b8afeef]
data.openstack_networking_network_v2.sharednet1: Reading...
data.openstack_networking_subnet_v2.sharednet1_subnet: Reading...
data.openstack_networking_secgroup_v2.allow_8000: Read complete after 3s [id=17a6d962-cc23-4551-ac0b-08b098f451d2]
data.openstack_networking_secgroup_v2.allow_http_80: Read complete after 3s [id=5df2db36-79f9-4b9b-b6c9-1ece762d83fb]
data.openstack_networking_secgroup_v2.allow_9001: Read complete after 3s [id=ad2912c0-caa9-49e0-b431-b9b3